In [1]:
import numpy as np

import itertools

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def nash_conditions_mem_two(p, b, c):
    c1  = p[1] < p[5]
    c2  = p[9] < p[5]
    c3  = p[13] < p[5]
    c4  = p[4] < 1
    c5  = p[8] < 1
    c6  = p[12] < 1
    c7  = (c / b) * p[2] < 1 - p[5]
    c8  = (c / b) * p[10] < 1 - p[5]
    c9  = (c / b) * p[14] < 1 - p[5]
    c12 = (c / b) * p[6] < 1 - p[5]
    c10 = 1 + (c / (b - c)) * p[11] > p[5]
    c11 = 1 + (c / (b - c)) * p[15] > p[5]
    c13 = (c / (b - c)) * p[7] < 1 - p[5]
    c14 = 1 + (c / (b - c)) * p[3] > p[5]
    
    return  {
"c1" : c1 ,
"c2" : c2 ,
"c3" : c3 ,
"c4" : c4 ,
"c5" : c5 ,
"c6" : c6 ,
"c7" : c7 ,
"c8" : c8 ,
"c9" : c9 ,
"c12" : c12,
"c10" : c10,
"c11" : c11,
"c13" : c13,
"c14" : c14,
}
    

In [4]:
def conditition_one(p2, p4, b, c):
    return p4 < 1 - (c / b)

def conditition_two(p2, p4, b, c):
    return p2 < p4

def conditition_three(p3, b, c):
    return p3 < 1

def condition_four(p2, p4, b, c):
    return  1 + p2 < (b / c) + (p4 * (c - b)) / c

def nash_conditions_reactive(p, b, c):
    p1, p2, p3, p4 = p
    
    c1 = conditition_one(p2, p4, b, c)
    c2 = conditition_two(p2, p4, b, c)
    c3 = conditition_three(p3, b, c)
    c4 = condition_four(p2, p4, b, c)
#     c5 = p1 == 1
    
    return {"c1": c1, "c2": c2, "c3": c3, "c4": c4}

In [5]:
nash_conditions_reactive((1, 0, 0, 1), 2, 1)

{'c1': False, 'c2': True, 'c3': True, 'c4': False}

In [6]:
two_bit_reactive_pure_set = list(itertools.product([0, 1], repeat=4))

memory_two_pure_set = list(itertools.product([0, 1], repeat=16))

In [7]:
is_nash = []

for strategy in two_bit_reactive_pure_set:
    checks = nash_conditions_reactive(strategy, 2, 1)
    
    if all(checks.values()):
        is_nash.append(strategy)

In [8]:
is_nash

[]

In [9]:
is_nash = []

for strategy in memory_two_pure_set:
    checks = nash_conditions_mem_two(strategy, 2, 1)
    
    if all(checks.values()):
        is_nash.append(strategy)

In [10]:
is_nash

[]

**Anti Press**

In [11]:
import os

os.chdir('..')

from importlib.machinery import SourceFileLoader

eq = SourceFileLoader("eq", "src/numerical_equilibria_n_bit_vs_n_bit.py").load_module()

from eq import *

from importlib.machinery import SourceFileLoader

main = SourceFileLoader("main", "src/main.py").load_module()

from main import *

In [12]:
def invariant_distributions(M):
    """Returns all the invariant distributions in case of absorbing states."""
    
    stationaries = []
    
    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    
    for index in np.where(eigenvalues == np.max(eigenvalues))[0]:
        
        eigenvectors_one = eigenvectors[:, index]

        stationary = eigenvectors_one / eigenvectors_one.sum()

        stationaries.append(stationary.real)
        
    return stationaries

In [13]:
def calculate_ss_for_mem_one(player, coplayer):
    """
    Invariant distribution for memory-one.
    """
    M = calculate_M(player, coplayer)
    ss = invariant_distributions(M)
    
    return ss



In [14]:
def calculate_ss_for_mem_two(player, coplayer):
    """
    Invariant distribution for memory-two.
    """
    M = calculate_M_memory_two(player, coplayer)
    ss = invariant_distributions(M)
    
    return ss

In [15]:
def expected_memory_one_strategy(player, coplayer, index=0):
    """
    Returns the expected memory-one strategy for a memory-two
    strategy from the player's prespective.
    
    This uses the method by Press and Dyson.
    """
    states = ['CC', 'CD', 'DC', 'DD']
    
    states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

    ss = calculate_ss_for_mem_two(player, coplayer)
    
    ss = ss[index]
    
    coplayer = [coplayer[0],  coplayer[2],  coplayer[1],  coplayer[3],
                 coplayer[8],  coplayer[10], coplayer[9],  coplayer[11],
                 coplayer[4],  coplayer[6],  coplayer[5],  coplayer[7],
                 coplayer[12], coplayer[14], coplayer[13], coplayer[15]]
    
    strategy = [(sum([coplayer[i] * ss[i] for i in range(16) if states_in_two_bits[i] == state]) 
                 / sum([ss[i] for i in range(16) if states_in_two_bits[i] == state]))
                for state in states]

    return [np.nan_to_num(strategy[i]) for i in [0, 2, 1, -1]]

In [16]:
# the two sets of strategies

set_of_memory_one_s = list(itertools.product([0, 1], repeat=4))

set_of_memory_two_s = list(itertools.product([0, 1], repeat=16))

In [60]:
b, c = 2, 1

for player in tqdm.tqdm_notebook(set_of_memory_one_s):

    for i, coplayer in enumerate(set_of_memory_one_s):
    

        mem_one_representation = expected_memory_one_strategy(player * 4, coplayer * 4, index=0)

        ss_estimated = calculate_ss_for_mem_one(player, mem_one_representation)

        ss = calculate_ss_for_mem_one(player, coplayer)


        check = False

        for ss1 in ss_estimated:
            for ss2 in ss:
                if (np.isclose(ss1, ss2, atol=10**-3).all()):
                    check = True

        # there are going to be some failures and I am just printing them. 
        if check == False:
            print(f"player: {player} coplayer:{coplayer}")

player: (0, 1, 0, 1) coplayer:(1, 0, 1, 1)
player: (0, 1, 1, 0) coplayer:(0, 1, 1, 1)
player: (0, 1, 1, 0) coplayer:(1, 0, 1, 1)
player: (1, 0, 1, 0) coplayer:(0, 1, 1, 1)



In [24]:
b, c = 2, 1

states = ['CC', 'CD', 'DC', 'DD']

states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

for player in tqdm.tqdm_notebook(set_of_memory_one_s):

    for i, coplayer in enumerate([np.random.random(16) for _ in range(10)]):
    
        mem_one_representation = expected_memory_one_strategy(player * 4, coplayer, index=0)
#         print(mem_one_representation)

        ss_estimated = calculate_ss_for_mem_one(player, mem_one_representation)

        ss = calculate_ss_for_mem_two(player * 4, coplayer)
        
        ss = [[sum([s[i] for i in range(16) if states_in_two_bits[i] == state])
                for state in states] for s in ss]


        check = False

        for ss1 in ss_estimated:
            for ss2 in ss:
                if (np.isclose(ss1, ss2, atol=10**-2).all()):
                    check = True

        # there are going to be some failures and I am just printing them. 
        if check == False:

            print(f"ss: {ss} est.:{ss_estimated}")

ss: [[0.01356408403394586, 0.02052100830174571, 0.2615455031443347, 0.7043694045199735]] est.:[array([ 0.3979477,  0.6020523, -0.       , -0.       ]), array([-0.        , -0.        ,  0.27077489,  0.72922511])]
ss: [[0.5319432064876651, 0.46805679351306895, -2.444895495827138e-13, -4.896813539507592e-13]] est.:[array([-0.        , -0.        ,  0.33299367,  0.66700633])]
ss: [[-0.025144982639419562, -0.014575956854761318, 0.5552704332377092, 0.4844505062564718]] est.:[array([0.63304098, 0.36695902, 0.        , 0.        ]), array([0.        , 0.        , 0.53405718, 0.46594282])]
ss: [[0.8071766644137416, 0.19282333558625855, 0.0, 0.0]] est.:[array([0., 0., 0., 1.])]



In [25]:
b, c = 2, 1

states = ['CC', 'CD', 'DC', 'DD']

states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

not_successful = []

for player in tqdm.tqdm_notebook(set_of_memory_one_s):

    for i, coplayer in enumerate(set_of_memory_two_s):
    

        mem_one_representation = expected_memory_one_strategy(player * 4, coplayer, index=0)

        ss_estimated = calculate_ss_for_mem_one(player, mem_one_representation)

        ss = calculate_ss_for_mem_two(player * 4, coplayer)
        
        ss = [[sum([s[i] for i in range(16) if states_in_two_bits[i] == state])
                for state in states] for s in ss]


        check = False

        for ss1 in ss_estimated:
            for ss2 in ss:
                if (np.isclose(ss1, ss2, atol=10**-3).all()):
                    check = True

        # there are going to be some failures and I am just printing them. 
        if check == False:
            not_successful.append((player, coplayer))
#             print(f"player: {player} coplayer:{coplayer}")

In [64]:
not_successful[0]

((0, 1, 0, 1), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0))

In [31]:
len(not_successful) / (len(memory_two_pure_set) * len(set_of_memory_one_s))

0.0101318359375

In [159]:
def expected_one_bit_reactive_strategy(player, coplayer, index=0):
    """
    Returns the expected memory-one strategy for a memory-two
    strategy from the player's prespective.
    
    This uses the method by Press and Dyson.
    """
    states = ['C', 'D']
    
    states_in_two_bits = ["".join(i[-1:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

    ss = calculate_ss_for_mem_two(player, coplayer)
    
    ss = ss[index]
    
    coplayer = [coplayer[0],  coplayer[2],  coplayer[1],  coplayer[3],
                 coplayer[8],  coplayer[10], coplayer[9],  coplayer[11],
                 coplayer[4],  coplayer[6],  coplayer[5],  coplayer[7],
                 coplayer[12], coplayer[14], coplayer[13], coplayer[15]]
    
    strategy = [(sum([coplayer[i] * ss[i] for i in range(16) if states_in_two_bits[i] == state]) 
                 / sum([ss[i] for i in range(16) if states_in_two_bits[i] == state]))
                for state in states]

    return [np.nan_to_num(strategy[i]) for i in [1, 0, 1, 0]]

In [160]:
set_of_one_bit_s = [(i, j, i, j) for i, j in list(itertools.product([0, 1], repeat=2))]

set_of_two_bit_s = [(i, j, i, j, k, l, k, l, 
                     i, j, i, j, k, l, k, l) for i,j, k, l in list(itertools.product([0, 1], repeat=4))]

In [161]:
b, c = 2, 1

states = ['CC', 'CD', 'DC', 'DD']

states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

not_successful_reactive = []

for player in tqdm.tqdm_notebook(set_of_one_bit_s):

    for i, coplayer in enumerate(set_of_two_bit_s):
        
        i, j, k, l = player
        
        player_reshape = (i, j, i, j, k, l, k, l, 
                          i, j, i, j, k, l, k, l)
    
        reactive_representation = expected_one_bit_reactive_strategy(player_reshape, coplayer, index=0)

        ss_estimated = calculate_ss_for_mem_one(player, reactive_representation)

        ss = calculate_ss_for_mem_two(player_reshape, coplayer)
        
        ss = [[sum([s[i] for i in range(16) if states_in_two_bits[i] == state])
                for state in states] for s in ss]


        check = False

        for ss1 in ss_estimated:
            for ss2 in ss:
                if (np.isclose(ss1, ss2, atol=10**-2).all()):
                    check = True

        # there are going to be some failures and I am just printing them. 
        if check == False:
            not_successful_reactive.append((player, coplayer))

#             print(f"ss: {ss} est.:{ss_estimated}")

In [162]:
not_successful_reactive

[((0, 1, 0, 1), (0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0)),
 ((0, 1, 0, 1), (0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1)),
 ((0, 1, 0, 1), (0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0)),
 ((0, 1, 0, 1), (1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0)),
 ((0, 1, 0, 1), (1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1)),
 ((0, 1, 0, 1), (1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0)),
 ((0, 1, 0, 1), (1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1)),
 ((0, 1, 0, 1), (1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0)),
 ((1, 0, 1, 0), (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1)),
 ((1, 0, 1, 0), (0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1)),
 ((1, 0, 1, 0), (0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0)),
 ((1, 0, 1, 0), (0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1)),
 ((1, 0, 1, 0), (1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0)),
 ((1, 0, 1, 0), (1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1)),
 ((1, 0, 1, 0), (1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 

In [163]:
b, c = 2, 1

states = ['CC', 'CD', 'DC', 'DD']

states_in_two_bits = ["".join(i[-2:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

not_successful_reactive = []

for player in tqdm.tqdm_notebook(set_of_one_bit_s):

    for i, coplayer in enumerate(set_of_memory_one_s):
        
        i, j, k, l = player
        
        player_reshape = (i, j, i, j, k, l, k, l, 
                          i, j, i, j, k, l, k, l)
    
        reactive_representation = expected_one_bit_reactive_strategy(player_reshape, coplayer * 4, index=0)
        

        ss_estimated = calculate_ss_for_mem_one(player, reactive_representation)

        ss = calculate_ss_for_mem_two(player_reshape, coplayer * 4)
        
        ss = [[sum([s[i] for i in range(16) if states_in_two_bits[i] == state])
                for state in states] for s in ss]


        check = False

        for ss1 in ss_estimated:
            for ss2 in ss:
                if (np.isclose(ss1 @ np.array([2, -1, 3, 0]), ss2 @ np.array([2, -1, 3, 0]), atol=10**-2).all()):
                    check = True

        # there are going to be some failures and I am just printing them. 
        if check == False:
            not_successful_reactive.append((player, coplayer))

#             print(f"ss: {ss} est.:{ss_estimated}")

In [164]:
not_successful_reactive

[((0, 0, 0, 0), (0, 0, 0, 1)),
 ((0, 0, 0, 0), (0, 0, 1, 1)),
 ((0, 0, 0, 0), (1, 0, 0, 1)),
 ((0, 0, 0, 0), (1, 0, 1, 1)),
 ((1, 0, 1, 0), (0, 0, 0, 1)),
 ((1, 0, 1, 0), (0, 1, 1, 1)),
 ((1, 0, 1, 0), (1, 0, 0, 0)),
 ((1, 0, 1, 0), (1, 0, 0, 1)),
 ((1, 0, 1, 0), (1, 1, 0, 0)),
 ((1, 0, 1, 0), (1, 1, 0, 1)),
 ((1, 0, 1, 0), (1, 1, 1, 0)),
 ((1, 0, 1, 0), (1, 1, 1, 1)),
 ((1, 1, 1, 1), (0, 0, 1, 0)),
 ((1, 1, 1, 1), (0, 0, 1, 1)),
 ((1, 1, 1, 1), (0, 1, 1, 0)),
 ((1, 1, 1, 1), (0, 1, 1, 1)),
 ((1, 1, 1, 1), (1, 0, 1, 0)),
 ((1, 1, 1, 1), (1, 0, 1, 1)),
 ((1, 1, 1, 1), (1, 1, 1, 0)),
 ((1, 1, 1, 1), (1, 1, 1, 1))]

In [165]:
player = (0, 1, 0, 1)

i, j, k, l = player

player_reshape = (i, j, i, j, k, l, k, l, 
                  i, j, i, j, k, l, k, l)

coplayer = (0, 0, 1, 0)

In [166]:
reactive_representation = expected_one_bit_reactive_strategy(player_reshape, coplayer * 4, index=0)

In [167]:
reactive_representation

[0.5000000000000001, 0.0, 0.5000000000000001, 0.0]

In [168]:
calculate_ss_for_mem_one(player, coplayer)

[array([ 0.33333333,  0.33333333, -0.        ,  0.33333333])]

In [169]:
calculate_ss_for_mem_one(player, reactive_representation)

[array([0.22222222, 0.44444444, 0.11111111, 0.22222222])]

In [170]:
ss = calculate_ss_for_mem_two(player_reshape, coplayer * 4)

In [171]:
ss = ss[0]

In [172]:
[sum([ss[i] for i in range(16) if states_in_two_bits[i] == state]) for state in states]

[0.3333333333333333, 0.3333333333333335, 0.0, 0.3333333333333333]

In [155]:
coplayer = coplayer * 4

states = ['C', 'D']

states_in_two_bits = ["".join(i[-1:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

ss = calculate_ss_for_mem_two(player_reshape, coplayer)

ss = ss[0]

coplayer = [coplayer[0],  coplayer[2],  coplayer[1],  coplayer[3],
             coplayer[8],  coplayer[10], coplayer[9],  coplayer[11],
             coplayer[4],  coplayer[6],  coplayer[5],  coplayer[7],
             coplayer[12], coplayer[14], coplayer[13], coplayer[15]]

strategy = [(sum([coplayer[i] * ss[i] for i in range(16) if states_in_two_bits[i] == state]) 
             / sum([ss[i] for i in range(16) if states_in_two_bits[i] == state]))
            for state in states]

In [156]:
strategy

[0.0, 0.5000000000000001]

In [158]:
[sum([ss[i] for i in range(16) if states_in_two_bits[i] == state]) for state in states]

[0.3333333333333333, 0.6666666666666667]

In [ ]:
    states = ['C', 'D']
    
    states_in_two_bits = ["".join(i[-1:]) for i in list(itertools.product(['C', 'D'], repeat=4))]

    ss = calculate_ss_for_mem_two(player, coplayer)
    
    ss = ss[index]
    
    coplayer = [coplayer[0],  coplayer[2],  coplayer[1],  coplayer[3],
                 coplayer[8],  coplayer[10], coplayer[9],  coplayer[11],
                 coplayer[4],  coplayer[6],  coplayer[5],  coplayer[7],
                 coplayer[12], coplayer[14], coplayer[13], coplayer[15]]
    
    strategy = [(sum([coplayer[i] * ss[i] for i in range(16) if states_in_two_bits[i] == state]) 
                 / sum([ss[i] for i in range(16) if states_in_two_bits[i] == state]))
                for state in states]



In [82]:
ss = calculate_ss_for_mem_one(player, coplayer)

ss

[array([-0.        ,  0.33333333,  0.33333333,  0.33333333])]

In [83]:
ss = calculate_ss_for_mem_one(player, reactive_representation)

ss

[array([0.11111111, 0.22222222, 0.22222222, 0.44444444])]

In [55]:
len(not_successful_reactive), (len(set_of_one_bit_s) * len(set_of_two_bit_s))

(23, 64)

In [56]:
not_successful_reactive[0]

((0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1))

In [57]:
i, j, k, l = (0, 0, 0, 0)

player_reshape = (i, j, i, j, k, l, k, l, 
                  i, j, i, j, k, l, k, l)

In [58]:
reactive_representation = expected_one_bit_reactive_strategy(player_reshape, coplayer, index=0)

In [59]:
reactive_representation

[1.0, 0.0, 1.0, 0.0]